# NBA Data Scrape

In [1]:
# Shoutout to my friends Ray Ford and James Ford for spending a whole weekend on ZOOM teaching me a faster scraping method :)

import requests
from bs4 import BeautifulSoup
import json
import csv
import logging
import re
import time
import datetime
from datetime import date
from dateutil.parser import parse

In [2]:
# Setup the csv writer
nbafile = open("./CSV/NBA_season_data.csv", "w", newline='', encoding='utf-8')
nbawriter = csv.writer(nbafile)
# set headings
nbawriter.writerow(['RK', 
                    'Team', 
                    'Year', 
                    'G', 
                    'W', 
                    'L', 
                    'Conf', 
                    'MP', 
                    'FG', 
                    'FGA', 
                    'FG%', 
                    '3P',
                    '3PA', 
                    '3P%', 
                    '2P', 
                    '2PA', 
                    '2P%', 
                    'FT', 
                    'FTA', 
                    'FT%', 
                    'ORB', 
                    'DRB', 
                    'AST', 
                    'STL', 
                    'BLK', 
                    'TOV', 
                    'PF', 
                    'PTS'])

103

## Select Start and End Year

In [3]:
# Define Year for URL
start_year = 1980
end_year = 2022

# Create model to scrape data

class NBAscrapy:
    runcount = 1
    rk = 1
    year = 1980
    
    # __init__ required to initialize the scrape tool
    def __init__(self):
        self.main()

    # starting URL according to start year above
    def main(self):
        url = "https://www.basketball-reference.com/leagues/NBA_"+str(start_year)+".html"
        self.getyearteamdata(url)
        url = "https://www.basketball-reference.com/leagues/NBA_"+str(start_year)+"_per_game.html"
        self.setup_player_csv(url)
#         self.getplayerdata(url)

    # build data for per-year
    def getyearteamdata(self, url):
        
        # running through each table by rank since website uses rank in each table used in ascending order
        # rank will reset to 1 once we move to a new year
        self.rk = 1
        print("--------------year: " + str(self.year) + " |" + " row: " + str(self.runcount) + "-----------------")
        
        # get data, total_pages
        page = requests.get(url)
        page.encoding = "UTF-8"
        pagecontent = str(page.content)
        pagecontent = pagecontent.replace('<!--',"")
        pagecontent = pagecontent.replace("-->", "")
        pagecontent = pagecontent.replace("\n", "")

        # print(pagecontent)
        soup = BeautifulSoup(pagecontent, 'html.parser')
        soup.encode('utf-8')

        # format material for page.json()
        
        # data will be pulled from 'Team Per Game Stats' table (avg game data)
        Team = ""
        Year = self.year
        G = ""
        MP = ""
        FG = ""
        FGA = ""
        FGpercent = ""
        ThreeP = ""
        ThreePA = ""
        ThreePpercent = ""
        TwoP = ""
        TwoPA = ""
        TwoPpercent = ""
        FT = ""
        FTA = ""
        FTpercent = ""
        ORB = ""
        DRB = ""
        TRB = ""
        AST = ""
        STL = ""
        BLK = ""
        TOV = ""
        PF = ""
        PTS = ""
        
        # data will be pulled from 'Conference Standings' table
        W = ""
        L = ""
        Conf = ""
        E_W_teams = []

        # for testing we wanted to split between East and West
        # This required pulling from the 'Conference Standings' table
        
        # If statement to Eastern Conference
        if soup.find('table', {'id':'divs_standings_E'}) is not None:
            if soup.find('table', {'id':'divs_standings_E'}).findAll('tr', {'class': 'full_table'}) is not None:
                E_team_trs = soup.find('table', {'id':'divs_standings_E'}).findAll('tr', {'class': 'full_table'})
                nI = 0
                
                # Loop through Eastern Conference, collect wins, lossess
                for E_team_tr in E_team_trs:
                    E_team = {
                        'index': 'East', # Index the data by 'East' Conference
                        'teamname': E_team_tr.find('a').text.strip(),
                        'W': E_team_tr.find('td',{'data-stat': 'wins'}).text.strip(),
                        'L': E_team_tr.find('td',{'data-stat': 'losses'}).text.strip()
                    }
                    E_W_teams.append(E_team.copy())
                    nI += 1
                    
        # If statement to Western Conference
        if soup.find('table', {'id':'divs_standings_W'}) is not None:
            if soup.find('table', {'id':'divs_standings_W'}).findAll('tr', {'class': 'full_table'}) is not None:
                W_team_trs = soup.find('table', {'id':'divs_standings_W'}).findAll('tr', {'class': 'full_table'})
                nI = 0
                
                # Loop through Western Conference, collect wins, lossess
                for W_team_tr in W_team_trs:
                    W_team = {
                        'index': 'West', # Index the data by 'West' Conference
                        'teamname': W_team_tr.find('a').text.strip(),
                        'W': W_team_tr.find('td',{'data-stat': 'wins'}).text.strip(),
                        'L': W_team_tr.find('td',{'data-stat': 'losses'}).text.strip()
                    }
                    E_W_teams.append(W_team.copy())
                    nI += 1
                    
        # If statement to collect data from league "Team Per Game Stats" table            
        if soup.find('table', {'id':'per_game-team'}) is not None:
            if soup.find('table', {'id':'per_game-team'}).findAll('tr') is not None:
                Team_stats_trs = soup.find('table', {'id':'per_game-team'}).findAll('tr')
                for Team_stats_tr in Team_stats_trs:
                    thhead = Team_stats_tr.find('th',{'scope': 'row'})
                    sheet = []
                    if thhead is not None:
                        Team = Team_stats_tr.find('td',{'data-stat': 'team'}).text.strip()
                        if Team != "League Average":
                            G = Team_stats_tr.find('td',{'data-stat': 'g'}).text.strip()
                            MP = Team_stats_tr.find('td',{'data-stat': 'mp'}).text.strip()
                            FG = Team_stats_tr.find('td',{'data-stat': 'fg'}).text.strip()
                            FGA = Team_stats_tr.find('td',{'data-stat': 'fga'}).text.strip()
                            FGpercent = Team_stats_tr.find('td',{'data-stat': 'fg_pct'}).text.strip()
                            ThreeP = Team_stats_tr.find('td',{'data-stat': 'fg3'}).text.strip()
                            ThreePA = Team_stats_tr.find('td',{'data-stat': 'fg3a'}).text.strip()
                            ThreePpercent = Team_stats_tr.find('td',{'data-stat': 'fg3_pct'}).text.strip()
                            TwoP = Team_stats_tr.find('td',{'data-stat': 'fg2'}).text.strip()
                            TwoPA = Team_stats_tr.find('td',{'data-stat': 'fg2a'}).text.strip()
                            TwoPpercent = Team_stats_tr.find('td',{'data-stat': 'fg2_pct'}).text.strip()
                            FT = Team_stats_tr.find('td',{'data-stat': 'ft'}).text.strip()
                            FTA = Team_stats_tr.find('td',{'data-stat': 'fta'}).text.strip()
                            FTpercent = Team_stats_tr.find('td',{'data-stat': 'ft_pct'}).text.strip()
                            ORB = Team_stats_tr.find('td',{'data-stat': 'orb'}).text.strip()
                            DRB = Team_stats_tr.find('td',{'data-stat': 'drb'}).text.strip()
                            TRB = Team_stats_tr.find('td',{'data-stat': 'trb'}).text.strip()
                            AST = Team_stats_tr.find('td',{'data-stat': 'ast'}).text.strip()
                            STL = Team_stats_tr.find('td',{'data-stat': 'stl'}).text.strip()
                            BLK = Team_stats_tr.find('td',{'data-stat': 'blk'}).text.strip()
                            TOV = Team_stats_tr.find('td',{'data-stat': 'tov'}).text.strip()
                            PF = Team_stats_tr.find('td',{'data-stat': 'pf'}).text.strip()
                            PTS = Team_stats_tr.find('td',{'data-stat': 'pts'}).text.strip()
                            
                            # Confirm the Win, Loss, and Conference
                            e_w_confirm = self.check_E_W(E_W_teams, Team)
                            if e_w_confirm != "error":
                                W = e_w_confirm['W']
                                L = e_w_confirm['L']
                                Conf = e_w_confirm['index']
                                
                            # append ALL team data
                            sheet.append(self.rk)
                            sheet.append(Team)
                            sheet.append(Year)
                            sheet.append(G)

                            sheet.append(W)
                            sheet.append(L)
                            sheet.append(Conf)

                            sheet.append(MP)
                            sheet.append(FG)
                            sheet.append(FGA)
                            sheet.append(FGpercent)
                            sheet.append(ThreeP)
                            sheet.append(ThreePA)
                            sheet.append(ThreePpercent)
                            sheet.append(TwoP)
                            sheet.append(TwoPA)
                            sheet.append(TwoPpercent)
                            sheet.append(FT)
                            sheet.append(FTA)
                            sheet.append(FTpercent)
                            sheet.append(ORB)
                            sheet.append(DRB)
                            sheet.append(AST)
                            sheet.append(STL)
                            sheet.append(BLK)
                            sheet.append(TOV)
                            sheet.append(PF)
                            sheet.append(PTS)
                            self.rk += 1
                            print(sheet)
                            nbawriter.writerow(sheet)
                            print("--------------------------------------------------------")
                            print(" ")
                            

        print("--------------------------------------------------------")
        print(" ")

        # move to next page to scrape if under year boundries
        if self.year <= (end_year-1):
            self.runcount += 1
            self.year += 1
            url = "https://www.basketball-reference.com/leagues/NBA_" + str(self.year) + ".html"
            self.getyearteamdata(url)
        else:
            nbafile.close()
            self.year = start_year
    def setup_player_csv(self,url):
        print(self.year)
        # Setup the csv writer
        nba_player_file = open("./CSV/player-info.csv", "w", newline='', encoding='utf-8')
        player_writer = csv.writer(nba_player_file)
        # set headings
        player_writer.writerow(['Name', 
                    'Team', 
                    'Year',   
                    'FG', 
                    'FGA', 
                    'FG%', 
                    '3P',
                    '3PA', 
                    '3P%', 
                    '2P', 
                    '2PA', 
                    '2P%', 
                    'FT', 
                    'FTA', 
                    'FT%', 
                    'ORB', 
                    'DRB', 
                    'AST', 
                    'STL', 
                    'BLK', 
                    'TOV', 
                    'PF', 
                    'PTS'])
        self.getplayerdata(url,player_writer)
    def getplayerdata(self,url,player_writer):
        
        page = requests.get(url)
        page.encoding = "UTF-8"
        pagecontent = str(page.content)
        pagecontent = pagecontent.replace('<!--',"")
        pagecontent = pagecontent.replace("-->", "")
        pagecontent = pagecontent.replace("\n", "")
        soup = BeautifulSoup(pagecontent, 'html.parser')
        soup.encode('utf-8')
        Name = ""
        Team = ""
        Year = self.year
        FG = ""
        FGA = ""
        FGpercent = ""
        ThreeP = ""
        ThreePA = ""
        ThreePpercent = ""
        TwoP = ""
        TwoPA = ""
        TwoPpercent = ""
        FT = ""
        FTA = ""
        FTpercent = ""
        ORB = ""
        DRB = ""
        TRB = ""
        AST = ""
        STL = ""
        BLK = ""
        TOV = ""
        PF = ""
        PTS = ""
        if soup.find('table', {'id':'per_game_stats'}) is not None:
            if soup.find('table', {'id':'per_game_stats'}).findAll('tr',{'class': 'full_table'}) is not None:
                player_stats_trs = soup.find('table', {'id':'per_game_stats'}).findAll('tr',{'class': 'full_table'})
                for player_stats_tr in player_stats_trs:
                    thhead = player_stats_tr.find('th',{'scope': 'row'})
                    sheet = []
                    if thhead is not None:
                        Name = player_stats_tr.find('td',{'data-stat':'player'}).text.strip()
                        Team = player_stats_tr.find('td',{'data-stat': 'team_id'}).text.strip()
                        FG = player_stats_tr.find('td',{'data-stat': 'fg_per_g'}).text.strip()
                        FGA = player_stats_tr.find('td',{'data-stat': 'fga_per_g'}).text.strip()
                        FGpercent = player_stats_tr.find('td',{'data-stat': 'fg_pct'}).text.strip()
                        ThreeP = player_stats_tr.find('td',{'data-stat': 'fg3_per_g'}).text.strip()
                        ThreePA = player_stats_tr.find('td',{'data-stat': 'fg3a_per_g'}).text.strip()
                        ThreePpercent = player_stats_tr.find('td',{'data-stat': 'fg3_pct'}).text.strip()
                        TwoP = player_stats_tr.find('td',{'data-stat': 'fg2_per_g'}).text.strip()
                        TwoPA = player_stats_tr.find('td',{'data-stat': 'fg2a_per_g'}).text.strip()
                        TwoPpercent = player_stats_tr.find('td',{'data-stat': 'fg2_pct'}).text.strip()
                        FT = player_stats_tr.find('td',{'data-stat': 'ft_per_g'}).text.strip()
                        FTA = player_stats_tr.find('td',{'data-stat': 'fta_per_g'}).text.strip()
                        FTpercent = player_stats_tr.find('td',{'data-stat': 'ft_pct'}).text.strip()
                        ORB = player_stats_tr.find('td',{'data-stat': 'orb_per_g'}).text.strip()
                        DRB = player_stats_tr.find('td',{'data-stat': 'drb_per_g'}).text.strip()
                        TRB = player_stats_tr.find('td',{'data-stat': 'trb_per_g'}).text.strip()
                        AST = player_stats_tr.find('td',{'data-stat': 'ast_per_g'}).text.strip()
                        STL = player_stats_tr.find('td',{'data-stat': 'stl_per_g'}).text.strip()
                        BLK = player_stats_tr.find('td',{'data-stat': 'blk_per_g'}).text.strip()
                        TOV = player_stats_tr.find('td',{'data-stat': 'tov_per_g'}).text.strip()
                        PF = player_stats_tr.find('td', {'data-stat': 'pf_per_g'}).text.strip()
                        PTS = player_stats_tr.find('td',{'data-stat': 'pts_per_g'}).text.strip()
                        sheet.append(Name)
                        sheet.append(Team)
                        sheet.append(Year)
                        sheet.append(FG)
                        sheet.append(FGA)
                        sheet.append(FGpercent)
                        sheet.append(ThreeP)
                        sheet.append(ThreePA)
                        sheet.append(ThreePpercent)
                        sheet.append(TwoP)
                        sheet.append(TwoPA)
                        sheet.append(TwoPpercent)
                        sheet.append(FT)
                        sheet.append(FTA)
                        sheet.append(FTpercent)
                        sheet.append(ORB)
                        sheet.append(DRB)
                        sheet.append(AST)
                        sheet.append(STL)
                        sheet.append(BLK)
                        sheet.append(TOV)
                        sheet.append(PF)
                        sheet.append(PTS)
                        print(sheet)
                        player_writer.writerow(sheet)
        if self.year <= (end_year-1):
            self.year += 1
            url = "https://www.basketball-reference.com/leagues/NBA_" + str(self.year) + "_per_game.html"
            self.getplayerdata(url,player_writer)
        
    def check_E_W(self, E_W_teams, teamname):
        t_name = "error"
        for E_W_team in E_W_teams:
            if E_W_team['teamname'] == teamname.replace("*",""):
                t_name = E_W_team
                break
        return t_name
NBAscrapy()

--------------year: 1980 | row: 1-----------------
[1, 'San Antonio Spurs*', 1980, '82', '41', '41', 'East', '240.9', '47.0', '94.4', '.498', '0.6', '2.5', '.252', '46.4', '91.9', '.505', '24.7', '30.8', '.801', '14.1', '30.7', '28.4', '9.4', '4.1', '19.4', '25.6', '119.4']
--------------------------------------------------------
 
[2, 'Los Angeles Lakers*', 1980, '82', '60', '22', 'West', '242.4', '47.5', '89.9', '.529', '0.2', '1.2', '.200', '47.3', '88.6', '.534', '19.8', '25.5', '.775', '13.2', '32.4', '29.4', '9.4', '6.7', '20.0', '21.8', '115.1']
--------------------------------------------------------
 
[3, 'Cleveland Cavaliers', 1980, '82', '37', '45', 'East', '243.0', '46.5', '98.1', '.474', '0.4', '2.3', '.193', '46.0', '95.8', '.481', '20.8', '26.9', '.772', '15.9', '29.0', '25.7', '9.3', '4.2', '16.7', '23.6', '114.1']
--------------------------------------------------------
 
[4, 'New York Knicks', 1980, '82', '39', '43', 'East', '241.2', '46.4', '93.6', '.496', '0.5', '2.

[1, 'Denver Nuggets*', 1982, '82', '46', '36', 'West', '241.8', '48.5', '93.4', '.520', '0.5', '1.8', '.268', '48.0', '91.5', '.525', '28.9', '36.3', '.796', '14.0', '29.8', '27.7', '8.1', '4.5', '17.9', '26.0', '126.5']
--------------------------------------------------------
 
[2, 'Los Angeles Lakers*', 1982, '82', '57', '25', 'West', '242.1', '47.8', '92.5', '.517', '0.2', '1.1', '.138', '47.6', '91.4', '.521', '18.9', '26.4', '.717', '15.3', '30.5', '28.7', '10.3', '6.3', '17.9', '24.4', '114.6']
--------------------------------------------------------
 
[3, 'San Antonio Spurs*', 1982, '82', '48', '34', 'West', '240.9', '45.1', '92.8', '.486', '0.8', '3.1', '.254', '44.3', '89.8', '.494', '22.1', '28.5', '.776', '15.3', '30.9', '27.5', '7.3', '6.8', '15.8', '27.0', '113.1']
--------------------------------------------------------
 
[4, 'Boston Celtics*', 1982, '82', '63', '19', 'East', '241.2', '44.6', '89.4', '.499', '0.6', '2.2', '.266', '44.0', '87.2', '.505', '22.2', '30.0', '.

[1, 'Denver Nuggets*', 1984, '82', '38', '44', 'West', '241.5', '48.0', '97.4', '.493', '0.9', '3.1', '.302', '47.0', '94.2', '.499', '26.8', '32.8', '.818', '13.8', '29.8', '30.3', '8.7', '4.3', '16.4', '27.8', '123.7']
--------------------------------------------------------
 
[2, 'San Antonio Spurs', 1984, '82', '37', '45', 'West', '243.7', '47.7', '94.2', '.506', '1.0', '3.2', '.300', '46.7', '91.0', '.514', '24.0', '31.8', '.755', '15.0', '30.8', '28.8', '8.4', '6.0', '17.6', '26.2', '120.3']
--------------------------------------------------------
 
[3, 'Detroit Pistons*', 1984, '82', '49', '33', 'East', '242.7', '46.3', '96.5', '.480', '0.4', '1.7', '.227', '45.9', '94.7', '.485', '24.1', '31.1', '.775', '17.4', '29.7', '27.5', '8.5', '5.1', '16.0', '26.5', '117.1']
--------------------------------------------------------
 
[4, 'Los Angeles Lakers*', 1984, '82', '54', '28', 'West', '241.2', '47.0', '88.4', '.532', '0.7', '2.8', '.257', '46.3', '85.7', '.540', '20.9', '27.7', '.7

[1, 'Los Angeles Lakers*', 1986, '82', '62', '20', 'West', '241.8', '46.8', '89.5', '.522', '1.7', '5.0', '.337', '45.1', '84.6', '.533', '22.1', '28.4', '.778', '13.4', '31.2', '29.7', '8.5', '5.1', '17.9', '24.8', '117.3']
--------------------------------------------------------
 
[2, 'Dallas Mavericks*', 1986, '82', '44', '38', 'West', '241.5', '44.3', '88.5', '.501', '1.7', '5.4', '.316', '42.6', '83.0', '.513', '25.0', '32.2', '.776', '12.9', '29.9', '25.7', '7.4', '4.5', '15.7', '21.1', '115.3']
--------------------------------------------------------
 
[3, 'Portland Trail Blazers*', 1986, '82', '40', '42', 'West', '242.1', '44.0', '88.8', '.496', '0.9', '3.4', '.269', '43.1', '85.4', '.505', '26.1', '34.1', '.765', '14.1', '28.2', '26.6', '10.5', '4.3', '18.6', '26.9', '115.1']
--------------------------------------------------------
 
[4, 'Denver Nuggets*', 1986, '82', '47', '35', 'West', '240.9', '45.2', '96.0', '.471', '0.9', '3.7', '.233', '44.3', '92.2', '.480', '23.5', '29

[1, 'Denver Nuggets*', 1988, '82', '54', '28', 'West', '241.2', '46.0', '97.1', '.474', '2.3', '6.9', '.342', '43.6', '90.2', '.484', '22.5', '27.9', '.804', '14.2', '29.8', '28.0', '10.1', '4.9', '14.5', '24.2', '116.7']
--------------------------------------------------------
 
[2, 'Portland Trail Blazers*', 1988, '82', '53', '29', 'West', '240.6', '44.6', '91.0', '.491', '1.4', '4.6', '.308', '43.2', '86.3', '.501', '25.4', '32.9', '.770', '15.3', '30.4', '28.1', '8.9', '4.2', '16.5', '25.5', '116.1']
--------------------------------------------------------
 
[3, 'Boston Celtics*', 1988, '82', '57', '25', 'East', '241.2', '43.9', '84.2', '.521', '3.3', '8.6', '.384', '40.6', '75.6', '.537', '22.5', '28.0', '.803', '11.3', '29.8', '29.9', '7.6', '5.1', '15.9', '22.0', '113.6']
--------------------------------------------------------
 
[4, 'San Antonio Spurs*', 1988, '82', '31', '51', 'West', '241.5', '45.2', '92.2', '.490', '1.6', '5.0', '.323', '43.6', '87.2', '.500', '21.6', '29.4'

...

['Jordan Nwora', 'MIL', 2021, '2.1', '4.5', '.459', '0.9', '2.1', '.452', '1.1', '2.4', '.466', '0.6', '0.8', '.760', '0.3', '1.7', '0.2', '0.5', '0.2', '0.8', '0.7', '5.7']
["Royce O\\'Neale", 'UTA', 2021, '2.5', '5.5', '.444', '1.5', '3.9', '.385', '1.0', '1.6', '.586', '0.5', '0.6', '.848', '1.2', '5.6', '2.5', '0.8', '0.5', '1.2', '2.6', '7.0']
['Semi Ojeleye', 'BOS', 2021, '1.6', '3.9', '.403', '1.0', '2.8', '.367', '0.6', '1.1', '.492', '0.4', '0.6', '.750', '0.6', '2.1', '0.7', '0.3', '0.0', '0.4', '1.0', '4.6']
['Jahlil Okafor', 'DET', 2021, '2.3', '3.8', '.618', '0.1', '0.3', '.222', '2.3', '3.4', '.656', '0.6', '0.9', '.708', '0.9', '1.6', '0.5', '0.2', '0.2', '0.8', '1.4', '5.4']
['Chuma Okeke', 'ORL', 2021, '3.0', '7.2', '.417', '1.1', '3.1', '.348', '1.9', '4.1', '.470', '0.7', '0.9', '.750', '0.9', '3.1', '2.2', '1.1', '0.5', '0.8', '1.1', '7.8']
['Josh Okogie', 'MIN', 2021, '1.7', '4.3', '.402', '0.5', '1.8', '.269', '1.2', '2.5', '.500', '1.4', '1.8', '.769', '1.1', '1.

['Tristan Thompson', 'BOS', 2021, '3.1', '6.0', '.518', '0.0', '0.1', '.000', '3.1', '6.0', '.523', '1.3', '2.2', '.592', '3.1', '5.0', '1.2', '0.4', '0.6', '1.1', '2.2', '7.6']
['Sindarius Thornwell', 'TOT', 2021, '0.7', '2.2', '.326', '0.3', '1.1', '.304', '0.4', '1.1', '.348', '0.2', '0.4', '.500', '0.1', '0.7', '1.0', '0.6', '0.1', '0.3', '0.7', '2.0']
['Matisse Thybulle', 'PHI', 2021, '1.6', '3.7', '.420', '0.7', '2.2', '.301', '0.9', '1.5', '.590', '0.1', '0.3', '.444', '0.5', '1.4', '1.0', '1.6', '1.1', '0.5', '2.0', '3.9']
['Killian Tillie', 'MEM', 2021, '1.1', '3.2', '.333', '0.6', '1.8', '.303', '0.5', '1.3', '.375', '0.5', '0.6', '.818', '0.3', '1.0', '0.4', '0.3', '0.4', '0.1', '1.1', '3.2']
['Xavier Tillman Sr.', 'MEM', 2021, '2.8', '5.1', '.559', '0.4', '1.1', '.338', '2.5', '4.0', '.620', '0.6', '0.9', '.642', '1.3', '3.1', '1.3', '0.7', '0.6', '0.8', '2.0', '6.6']
['Anthony Tolliver', 'PHI', 2021, '0.4', '1.5', '.235', '0.4', '1.3', '.286', '0.0', '0.3', '.000', '0.5', 

['Precious Achiuwa', 'TOR', 2022, '3.4', '8.8', '.393', '0.4', '1.3', '.350', '3.0', '7.5', '.400', '0.9', '1.7', '.556', '2.1', '6.1', '1.6', '0.6', '0.6', '1.0', '2.3', '8.3']
['Steven Adams', 'MEM', 2022, '2.6', '5.3', '.495', '0.0', '0.0', '', '2.6', '5.3', '.495', '1.8', '2.6', '.696', '3.7', '4.4', '2.8', '0.8', '0.7', '1.5', '1.8', '7.0']
['Bam Adebayo', 'MIA', 2022, '6.9', '13.4', '.516', '0.0', '0.1', '.000', '6.9', '13.4', '.519', '5.1', '6.6', '.774', '2.6', '7.6', '2.8', '1.2', '0.4', '2.9', '3.3', '19.0']
['Santi Aldama', 'MEM', 2022, '1.1', '3.3', '.348', '0.1', '1.4', '.100', '1.0', '1.9', '.538', '0.3', '0.4', '.667', '0.7', '1.1', '0.7', '0.0', '0.0', '0.0', '0.4', '2.7']
['LaMarcus Aldridge', 'BRK', 2022, '5.8', '10.1', '.575', '0.5', '1.3', '.391', '5.3', '8.8', '.601', '1.6', '2.0', '.778', '1.6', '4.3', '0.9', '0.3', '1.2', '0.7', '1.7', '13.6']
['Nickeil Alexander-Walker', 'NOP', 2022, '5.4', '14.1', '.379', '2.1', '6.8', '.301', '3.3', '7.3', '.452', '1.7', '2.2'

['Evan Fournier', 'NYK', 2022, '4.6', '11.1', '.417', '2.4', '6.4', '.379', '2.2', '4.6', '.470', '1.2', '1.5', '.778', '0.6', '2.4', '1.8', '0.9', '0.2', '1.2', '1.6', '12.8']
["De\\'Aaron Fox", 'SAC', 2022, '7.5', '17.4', '.429', '1.1', '4.3', '.247', '6.4', '13.2', '.488', '3.9', '5.5', '.721', '0.6', '2.7', '5.6', '1.3', '0.5', '3.1', '2.8', '19.9']
['Daniel Gafford', 'WAS', 2022, '3.7', '5.7', '.647', '0.0', '0.0', '', '3.7', '5.7', '.647', '1.1', '1.8', '.630', '1.8', '3.1', '0.8', '0.6', '1.7', '1.1', '2.5', '8.5']
['Danilo Gallinari', 'ATL', 2022, '2.5', '6.3', '.396', '1.2', '3.3', '.365', '1.3', '3.1', '.429', '1.8', '2.0', '.906', '0.7', '3.1', '1.0', '0.6', '0.3', '0.5', '1.1', '8.0']
['Darius Garland', 'CLE', 2022, '6.8', '15.2', '.448', '2.5', '7.3', '.347', '4.3', '7.9', '.541', '2.1', '2.4', '.854', '0.4', '2.1', '6.8', '1.3', '0.1', '3.6', '1.6', '18.2']
['Marcus Garrett', 'MIA', 2022, '0.0', '0.3', '.000', '0.0', '0.0', '', '0.0', '0.3', '.000', '0.0', '0.0', '', '0.0

['Aaron Wiggins', 'OKC', 2022, '1.0', '2.4', '.412', '0.7', '2.1', '.333', '0.3', '0.3', '1.000', '0.3', '0.7', '.400', '0.1', '1.4', '1.3', '0.4', '0.1', '1.0', '1.3', '3.0']
['Andrew Wiggins', 'GSW', 2022, '7.1', '14.3', '.496', '1.9', '5.2', '.362', '5.2', '9.1', '.573', '2.9', '3.9', '.757', '1.1', '3.3', '1.6', '0.8', '0.5', '1.8', '1.5', '19.1']
['Grant Williams', 'BOS', 2022, '2.5', '5.1', '.500', '1.4', '3.5', '.409', '1.1', '1.6', '.700', '0.8', '0.9', '.882', '0.6', '2.7', '1.2', '0.3', '0.4', '0.5', '1.7', '7.3']
['Kenrich Williams', 'OKC', 2022, '2.6', '5.8', '.438', '0.7', '2.1', '.316', '1.9', '3.7', '.507', '0.8', '1.4', '.538', '0.9', '2.5', '1.8', '0.7', '0.2', '0.8', '1.7', '6.6']
['Lou Williams', 'ATL', 2022, '2.4', '5.9', '.402', '0.6', '1.6', '.364', '1.8', '4.3', '.417', '0.9', '0.9', '.923', '0.1', '1.1', '1.5', '0.4', '0.1', '0.8', '0.9', '6.1']
['Patrick Williams', 'CHI', 2022, '2.6', '4.6', '.565', '0.6', '1.2', '.500', '2.0', '3.4', '.588', '0.8', '0.8', '1.0